### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import copy
import random
from PIL import Image
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.3 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# Colabのローカルファイルに自作データをコピー（初めからローカルファイルにアップロードする場合は不要）
!cp -r "/content/drive/MyDrive/HandSigns/" "/content/HandSigns/"

In [5]:
# 自作データをリサイズして、ファイル構造を保ったまま新しいフォルダに保存する関数を定義
def resize_images(base_dir, new_base_dir, image_size=(32, 32)):
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        new_folder_path = os.path.join(new_base_dir, folder)
        # リサイズした自作データを保存する新しいフォルダを作成
        os.makedirs(new_folder_path, exist_ok=True)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            # 拡張子の確認
            if file_path.endswith(('.jpg', '.jpeg')):
                # 画像を展開
                img = Image.open(file_path)
                # 画像をリサイズ
                img_resized = img.resize(image_size)
                # リサイズした画像を新しいフォルダに保存
                img_resized.save(os.path.join(new_folder_path, filename))

# 学習用の自作データのディレクトリを設定
train_base_dir = '/content/HandSigns/train'
# テスト用の自作データのディレクトリを設定
test_base_dir = '/content/HandSigns/test'

# リサイズした学習用の自作データを保存するディレクトリを設定
new_train_base_dir = '/content/HandSigns/resized_train_32'
# リサイズしたテスト用の自作データを保存するディレクトリを設定
new_test_base_dir = '/content/HandSigns/resized_test_32'
# リサイズする大きさを設定
image_size = (32, 32)

# 学習用の自作データをリサイズ
resize_images(train_base_dir, new_train_base_dir, image_size)
# テスト用の自作データをリサイズ
resize_images(test_base_dir, new_test_base_dir, image_size)

In [6]:
# 平均値と標準偏差を計算するためのHandSignsの前処理を定義
pre_transform_handsigns = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのHandSignsデータセットの前処理を定義
pre_train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=pre_transform_handsigns)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_handsigns = 0.0
pre_std_handsigns = 0.0
pre_total_samples_handsigns = len(pre_train_dataset_handsigns)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_handsigns:
    pre_image, _ = data
    pre_mean_handsigns += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_handsigns += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_handsigns /= pre_total_samples_handsigns
pre_std_handsigns /= pre_total_samples_handsigns

print("データセット全体の平均値: ", pre_mean_handsigns)
print("データセット全体の標準偏差: ", pre_std_handsigns)

データセット全体の平均値:  tensor([0.6954, 0.6754, 0.6450])
データセット全体の標準偏差:  tensor([0.1331, 0.1565, 0.1689])


In [7]:
# 学習用のHandSignsデータセットの前処理を定義
train_transform_handsigns = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# テスト用のHandSignsデータセットの前処理を定義
test_transform_handsigns = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# 学習用のHandSignsデータセットの読み込み
train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=train_transform_handsigns)
# テスト用のHandSignsデータセットの読み込み
test_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_test_32", transform=test_transform_handsigns)

# 学習用のHandSignsデータローダーを作成
train_loader_handsigns = torch.utils.data.DataLoader(dataset=train_dataset_handsigns, batch_size=16, shuffle=True, num_workers=2)
# テスト用のHandSignsデータローダーを作成
test_loader_handsigns = torch.utils.data.DataLoader(dataset=test_dataset_handsigns, batch_size=16, shuffle=False, num_workers=2)

In [8]:
# 学習データのクラス名とラベルの対応を確認
print(train_dataset_handsigns.class_to_idx)
# テストデータのクラス名とラベルの対応を確認
print(test_dataset_handsigns.class_to_idx)

{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}
{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}


In [9]:
# イテレータを作成
data_iter = iter(train_loader_handsigns)

# データを1バッチ分取得
images, labels = next(data_iter)

# 取得したバッチの画像とラベルの情報を確認
print("画像の形状:", images.shape)  # (バッチサイズ, チャンネル数, 高さ, 幅)
print("ラベル:", labels)

画像の形状: torch.Size([16, 3, 32, 32])
ラベル: tensor([3, 7, 4, 1, 3, 5, 6, 9, 9, 5, 3, 0, 4, 0, 2, 1])


## モデルの実装

In [10]:
# 1×1のサブネットワーク獲得用の畳み込みを定義
def supermaskconv1x1(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3のサブネットワーク獲得用の畳み込みを定義
def supermaskconv3x3(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [11]:
# サブネットワーク獲得用のバッチ正則化として、非アフィン正規化を使用する（学習可能なパラメータを使用しない）
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

In [12]:
# 各重みにスコアを付与したスコアをソートしてtop k%を使用することでサブネットワークを獲得する
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # スコアを複製する
        out = scores.clone()
        # スコアを昇順でソートする
        _, idx = scores.flatten().sort()
        # top k%以下のスコアの数
        j = int((1 - k) * scores.numel())
        # flat_outとoutは同じメモリを参照する（flat_outを変更するとoutにも影響する）
        flat_out = out.flatten()
        # top k%の要素を1にする
        flat_out[idx[j:]] = 1
        # top k%以外の要素を0にする
        flat_out[idx[:j]] = 0
        return out

    @staticmethod
    def backward(ctx, g):
        # 逆伝播時に勾配gをそのまま伝える
        return g, None

In [13]:
# サブネットワーク獲得用の畳み込みを定義
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 重みと同じ形状のスコアを用意
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        # スコアを一様分布で初期化
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))
        # 重みの勾配を無効化する
        self.weight.requires_grad = False

    def _init_conv(self):
        # 重みを一様分布で初期化
        if self.init == 'kaiming_uniform':
            nn.init.kaiming_uniform_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを正規分布で初期化
        elif self.init == 'kaiming_normal':
            nn.init.kaiming_normal_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを符号つき定数で初期化
        elif self.init == 'signed_constant':
            fan = nn.init._calculate_correct_fan(self.weight, mode='fan_out')
            gain = nn.init.calculate_gain('relu')
            std = gain / math.sqrt(fan)
            self.weight.data = self.weight.data.sign() * std

    def set_init(self, init):
        # 初期化手法を設定
        self.init = init

    def set_prune_rate(self, prune_rate):
        # 刈り込み率を設定
        self.prune_rate = prune_rate

    @property
    def clamped_scores(self):
        # スコアとして非負の値を返すようにする（重要度を表す）
        return self.scores.abs()

    def forward(self, x):
        # サブネットワークを獲得
        subnet = GetSubnet.apply(self.clamped_scores, 1 - self.prune_rate)
        # サブネットワークでマスク
        w = self.weight * subnet
        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [14]:
# サブネットワーク獲得用のResidual Blocksを定義
class SupermaskBuildingBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.bn1 = NonAffineBatchNorm(in_channels)
        self.conv1 = supermaskconv3x3(in_channels, out_channels, stride)
        self.bn2 = NonAffineBatchNorm(out_channels)
        self.conv2 = supermaskconv3x3(out_channels, out_channels)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != out_channels or stride > 1:
            self.shortcut = supermaskconv1x1(in_channels, out_channels, stride)
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        # 残差写像と恒等写像の要素毎の和を計算
        out += self.shortcut(x)
        return out

In [15]:
class SupermaskWideResNet(nn.Module):
    def __init__(self, block, depth, k, num_classes=10):
        super().__init__()
        assert (depth - 4) % 6 == 0, "depth should be 6n + 4"
        n = (depth - 4) // 6
        channels = [16, 16 * k, 32 * k, 64 * k]
        self.conv1 = supermaskconv3x3(3, channels[0])
        # Residual Blocks（1)
        self.layer1 = self._make_layer(block, channels[0], channels[1], n)
        # Residual Blocks（2）
        self.layer2 = self._make_layer(block, channels[1], channels[2], n, stride=2)
        # Residual Blocks（3）
        self.layer3 = self._make_layer(block, channels[2], channels[3], n, stride=2)
        self.bn = NonAffineBatchNorm(channels[3])
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(channels[3], num_classes)

        # 重みの初期化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 初期化手法を設定
                m.set_init(init)
                # 刈り込み率を設定
                m.set_prune_rate(prune_rate)
                # 重みを初期化
                m._init_conv()

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, in_channels, out_channels, blocks, stride=1):
        layers = []
        # 最初の Residual Block（stride=stride）
        layers.append(block(in_channels, out_channels, stride))
        # 残りの Residual Block（stride=1）
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

## 学習と評価（100 epochs, init=kaiming_uniform）

### prune_rate=0.999

In [16]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.999_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.999_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 11.0 %, Loss: 2.3556
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 14.7 %, Loss: 2.2715
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 25.2 %, Loss: 2.0170
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 25.7 %, Loss: 1.9384
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.2 %, Loss: 2.0662
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 32.7 %, Loss: 1.2611
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 35.6 %, Loss: 1.5516
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 35.9 %, Loss: 1.4276
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 37.8 %, Loss: 2.5165
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.4 %, Loss: 1.8307
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 43.5 %, Loss: 1.1343
Epoch [11/50], Lear

In [17]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [18]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [19]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [20]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 59.0 %


In [21]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [22]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.995_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.995_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.1 %, Loss: 2.2326
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 25.6 %, Loss: 2.2115
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 34.5 %, Loss: 1.9586
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 45.9 %, Loss: 1.3850
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 47.9 %, Loss: 1.7803
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 52.0 %, Loss: 2.0324
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 55.1 %, Loss: 1.3911
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 58.7 %, Loss: 1.2975
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 60.9 %, Loss: 0.8733
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 60.4 %, Loss: 1.1043
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 58.0 %, Loss: 1.3189
Epoch [11/50], Lear

In [23]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [24]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [25]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [26]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.0 %


In [27]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [28]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.99_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.99_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 17.1 %, Loss: 2.1661
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 24.9 %, Loss: 2.2266
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 39.6 %, Loss: 1.5521
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 52.0 %, Loss: 1.1774
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 59.2 %, Loss: 0.7397
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 60.8 %, Loss: 0.6776
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 57.5 %, Loss: 1.7969
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 64.6 %, Loss: 2.3607
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 65.3 %, Loss: 1.6955
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 64.4 %, Loss: 1.3339
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 70.9 %, Loss: 2.4059
Epoch [11/50], Lear

In [29]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [30]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [31]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [32]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 85.0 %


In [33]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [34]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.95_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.95_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.1 %, Loss: 2.3428
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 20.7 %, Loss: 2.2705
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 41.0 %, Loss: 1.7888
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 51.4 %, Loss: 1.2176
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 58.3 %, Loss: 1.3319
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 61.0 %, Loss: 1.5627
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 62.4 %, Loss: 1.6452
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 64.8 %, Loss: 0.7591
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 66.8 %, Loss: 0.7547
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 65.8 %, Loss: 1.0968
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 66.7 %, Loss: 1.7497
Epoch [11/50], Lear

In [35]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [36]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [37]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [38]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 87.0 %


In [39]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [40]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.9_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.9_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 13.3 %, Loss: 2.1887
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 25.5 %, Loss: 2.2162
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 40.7 %, Loss: 1.4023
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 55.9 %, Loss: 1.2182
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 59.6 %, Loss: 0.6903
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 64.8 %, Loss: 1.3975
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 65.8 %, Loss: 1.7481
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 64.4 %, Loss: 1.1170
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 67.4 %, Loss: 1.1762
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 71.1 %, Loss: 1.3206
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 72.6 %, Loss: 1.6867
Epoch [11/50], Lear

In [41]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [42]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [43]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [44]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.0 %


In [45]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [46]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.7_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.7_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.1 %, Loss: 2.2238
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 27.4 %, Loss: 2.2502
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 41.4 %, Loss: 1.8784
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 54.7 %, Loss: 1.2482
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 56.1 %, Loss: 1.4110
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 62.2 %, Loss: 1.9865
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 62.0 %, Loss: 0.7834
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 68.5 %, Loss: 0.6342
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 69.4 %, Loss: 0.9412
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 71.2 %, Loss: 0.3773
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 70.8 %, Loss: 0.8319
Epoch [11/50], Lear

In [47]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [48]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [49]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [50]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 86.0 %


In [51]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [52]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.5_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.5_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.6 %, Loss: 2.2414
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 20.6 %, Loss: 2.1862
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 39.4 %, Loss: 1.3055
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 59.5 %, Loss: 0.9427
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 59.7 %, Loss: 1.6106
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 60.4 %, Loss: 0.7420
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 65.0 %, Loss: 0.5195
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 67.1 %, Loss: 1.6537
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 68.3 %, Loss: 1.0543
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 70.2 %, Loss: 0.8011
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 72.3 %, Loss: 0.5138
Epoch [11/50], Lear

In [53]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [54]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [55]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [56]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 89.0 %


In [57]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=kaiming_normal）

### prune_rate=0.999

In [58]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.999_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.999_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 12.9 %, Loss: 2.2783
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 16.4 %, Loss: 2.1581
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 21.4 %, Loss: 2.0116
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 29.8 %, Loss: 2.1605
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 35.0 %, Loss: 1.7792
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 34.6 %, Loss: 1.3753
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 36.0 %, Loss: 2.5086
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 35.6 %, Loss: 2.5267
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.4 %, Loss: 2.0190
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 42.2 %, Loss: 1.8656
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 39.7 %, Loss: 1.6853
Epoch [11/50], Lear

In [59]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [60]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [61]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [62]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 63.0 %


In [63]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [64]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.995_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.995_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 12.1 %, Loss: 2.3028
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 20.3 %, Loss: 2.1820
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 33.9 %, Loss: 2.1227
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 42.7 %, Loss: 1.6912
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 46.5 %, Loss: 1.3162
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 53.1 %, Loss: 1.1858
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 53.7 %, Loss: 2.1313
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 57.5 %, Loss: 1.6273
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 55.8 %, Loss: 1.7905
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 56.9 %, Loss: 2.2290
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 59.3 %, Loss: 1.2015
Epoch [11/50], Lear

In [65]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [66]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [67]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [68]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 81.0 %


In [69]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [70]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.99_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.99_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 12.8 %, Loss: 2.2755
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 24.8 %, Loss: 2.2159
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 37.3 %, Loss: 1.6395
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 51.5 %, Loss: 1.8506
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 51.1 %, Loss: 1.2633
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 54.6 %, Loss: 2.2236
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 56.6 %, Loss: 1.2103
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 63.2 %, Loss: 0.6266
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 58.5 %, Loss: 0.7785
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 62.6 %, Loss: 1.4964
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 66.7 %, Loss: 0.4929
Epoch [11/50], Lear

In [71]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [72]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [73]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [74]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 75.0 %


In [75]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [76]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.95_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.95_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 7.9 %, Loss: 2.2513
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 20.2 %, Loss: 2.1191
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 40.9 %, Loss: 1.4720
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 54.7 %, Loss: 0.8214
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 60.9 %, Loss: 0.9439
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 59.4 %, Loss: 1.3865
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 63.4 %, Loss: 1.7270
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 68.3 %, Loss: 1.5527
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 67.0 %, Loss: 1.9623
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 68.0 %, Loss: 1.5890
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 73.7 %, Loss: 0.3357
Epoch [11/50], Learn

In [77]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [78]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [79]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [80]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.0 %


In [81]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [82]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.9_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.9_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 16.7 %, Loss: 2.2916
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 27.7 %, Loss: 2.1635
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 43.5 %, Loss: 1.7982
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 56.4 %, Loss: 1.6203
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 60.0 %, Loss: 1.4700
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 63.0 %, Loss: 1.2516
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 62.4 %, Loss: 0.2554
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 65.7 %, Loss: 1.3673
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 66.5 %, Loss: 1.1690
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 68.3 %, Loss: 1.6153
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 71.4 %, Loss: 1.4519
Epoch [11/50], Lear

In [83]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [84]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [85]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [86]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 84.0 %


In [87]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [92]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.7_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.7_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.0 %, Loss: 2.2480
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 22.4 %, Loss: 2.2903
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 45.8 %, Loss: 1.7439
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 54.4 %, Loss: 1.6700
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 61.1 %, Loss: 1.3511
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 62.7 %, Loss: 1.0437
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 65.3 %, Loss: 0.5259
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 68.6 %, Loss: 1.5957
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 73.0 %, Loss: 0.5855
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 70.7 %, Loss: 0.6820
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 71.0 %, Loss: 0.6161
Epoch [11/50], Lear

In [93]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [94]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [95]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [96]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 81.0 %


In [97]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [98]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.5_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.5_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.5 %, Loss: 2.2787
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 27.9 %, Loss: 2.2102
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 41.8 %, Loss: 1.4520
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 52.8 %, Loss: 1.2068
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 64.4 %, Loss: 1.1264
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 64.6 %, Loss: 2.2898
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 65.6 %, Loss: 0.5181
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 71.1 %, Loss: 0.8143
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 72.1 %, Loss: 1.8367
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 75.7 %, Loss: 0.6405
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 73.3 %, Loss: 1.7329
Epoch [11/50], Lear

In [99]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [100]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [101]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [102]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.0 %


In [103]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=signed_constant）

### prune_rate=0.999

In [104]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.999_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.999_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 11.4 %, Loss: 2.3257
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 14.5 %, Loss: 2.3157
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 23.5 %, Loss: 2.2275
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 31.7 %, Loss: 1.4331
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 37.3 %, Loss: 1.8470
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 39.8 %, Loss: 2.0902
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 37.8 %, Loss: 1.4546
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 38.7 %, Loss: 1.6280
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 44.3 %, Loss: 1.9248
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 43.3 %, Loss: 2.4547
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 49.5 %, Loss: 1.2127
Epoch [11/50], Lear

In [105]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [106]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [107]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [108]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 72.0 %


In [109]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [110]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.995_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.995_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.7 %, Loss: 2.2526
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 19.3 %, Loss: 2.2313
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 37.3 %, Loss: 1.8772
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 47.4 %, Loss: 1.5783
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 52.5 %, Loss: 1.2925
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 56.4 %, Loss: 1.0626
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 57.9 %, Loss: 1.8460
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 60.8 %, Loss: 0.9956
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 60.1 %, Loss: 1.5115
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 63.5 %, Loss: 0.7786
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 68.9 %, Loss: 1.7461
Epoch [11/50], Lear

In [111]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [112]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [113]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [114]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 85.0 %


In [115]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [116]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.99_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.99_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 17.6 %, Loss: 2.2915
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 29.1 %, Loss: 2.1678
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 37.2 %, Loss: 1.7900
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 53.1 %, Loss: 1.3972
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 57.0 %, Loss: 1.5406
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 62.4 %, Loss: 0.5340
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 61.9 %, Loss: 1.7171
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 67.2 %, Loss: 0.6327
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 67.0 %, Loss: 1.7356
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 65.7 %, Loss: 0.5745
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 68.3 %, Loss: 0.7126
Epoch [11/50], Lear

In [117]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [118]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [119]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [120]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.0 %


In [121]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [122]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.95_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.95_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.5 %, Loss: 2.2381
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 23.0 %, Loss: 2.2129
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 43.5 %, Loss: 1.9391
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 55.4 %, Loss: 1.2686
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 62.5 %, Loss: 0.5647
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 63.6 %, Loss: 1.3266
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 61.4 %, Loss: 2.3870
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 66.7 %, Loss: 1.9649
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 71.2 %, Loss: 0.5711
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 72.0 %, Loss: 0.5900
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 74.1 %, Loss: 1.8351
Epoch [11/50], Lear

In [123]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [124]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [125]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [126]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 84.0 %


In [127]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [128]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.9_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.9_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.4 %, Loss: 2.2874
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 24.0 %, Loss: 2.1638
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 40.8 %, Loss: 1.7649
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 52.4 %, Loss: 0.9643
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 60.6 %, Loss: 0.7434
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 66.2 %, Loss: 1.4542
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 63.7 %, Loss: 0.7922
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 66.8 %, Loss: 1.4499
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 69.6 %, Loss: 2.2684
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 71.3 %, Loss: 0.6516
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 72.8 %, Loss: 0.2352
Epoch [11/50], Lear

In [129]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [130]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [131]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [132]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 84.0 %


In [133]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [134]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.7_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.7_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 15.1 %, Loss: 2.2950
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 23.8 %, Loss: 2.1667
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 38.7 %, Loss: 1.4350
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 52.5 %, Loss: 1.0676
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 60.2 %, Loss: 0.9464
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 65.2 %, Loss: 1.2024
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 66.2 %, Loss: 0.3819
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 67.7 %, Loss: 1.8556
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 69.8 %, Loss: 0.9325
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 69.7 %, Loss: 0.1598
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 73.9 %, Loss: 1.4551
Epoch [11/50], Lear

In [135]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [136]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [137]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [138]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 86.0 %


In [139]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [140]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.5_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR100_100epochs_pr0.5_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 12.7 %, Loss: 2.3092
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 24.1 %, Loss: 2.2553
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 41.0 %, Loss: 1.5336
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 54.7 %, Loss: 1.4949
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 60.4 %, Loss: 1.4781
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 61.9 %, Loss: 1.0885
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 61.8 %, Loss: 0.8228
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 69.5 %, Loss: 0.6067
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 63.2 %, Loss: 1.5003
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 67.4 %, Loss: 0.7392
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 71.9 %, Loss: 2.1804
Epoch [11/50], Lear

In [141]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [142]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [143]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [144]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 88.0 %


In [145]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()